In [ ]:
# Montar Google Drive para guardar modelos y checkpoints
from google.colab import drive
drive.mount('/content/drive')

# Instalar librerías necesarias
!pip install -q kaggle

# Subir tu archivo kaggle.json (credenciales de API)
from google.colab import files
files.upload() # Sube el archivo kaggle.json

# Mover kaggle.json a la ubicación correcta
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Descargar el dataset Food-101
!kaggle datasets download -d kmader/food41
!unzip -q food41.zip -d food101

Mounted at /content/drive


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/kmader/food41
License(s): copyright-authors
100% 5.29G/5.30G [01:14<00:00, 154MB/s]
100% 5.30G/5.30G [01:14<00:00, 76.8MB/s]


In [ ]:
import os
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Food101 ya viene con carpetas train y test organizadas
base_dir = '/content/food101/images'

# Verificamos que haya subcarpetas por clase
print("Clases:", len(os.listdir(base_dir)))

Clases: 101


In [ ]:
# Rutas base
images_dir = '/content/food101/images'
meta_dir = '/content/food101/meta/meta'
output_base = '/content/food101_split'

# Crear carpetas /train y /test con subcarpetas por clase
for split in ['train', 'test']:
    for class_name in os.listdir(images_dir):
        os.makedirs(os.path.join(output_base, split, class_name), exist_ok=True)

# Leer archivos de división
with open(os.path.join(meta_dir, 'train.txt'), 'r') as f:
    train_list = [line.strip() for line in f]

with open(os.path.join(meta_dir, 'test.txt'), 'r') as f:
    test_list = [line.strip() for line in f]

# Copiar imágenes a sus carpetas correspondientes
for item in train_list:
    src = os.path.join(images_dir, item + '.jpg')
    dst = os.path.join(output_base, 'train', item.split('/')[0])
    shutil.copy(src, dst)

for item in test_list:
    src = os.path.join(images_dir, item + '.jpg')
    dst = os.path.join(output_base, 'test', item.split('/')[0])
    shutil.copy(src, dst)

print("✅ Imágenes reorganizadas correctamente en /food101_split/train y /test.")


✅ Imágenes reorganizadas correctamente en /food101_split/train y /test.


In [ ]:
# Ruta destino en tu Drive (puedes cambiar el nombre si quieres)
destination = '/content/drive/MyDrive/food101_split_backup'

# Copiar carpeta completa a Drive
!cp -r /content/food101_split "$destination"

print(f"✅ Dataset copiado correctamente a: {destination}")

✅ Dataset copiado correctamente a: /content/drive/MyDrive/food101_split_backup


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam

# Cargar modelo base sin la capa superior (Top = False)
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
base_model.trainable = False  # Congelar capas base

# Agregar capas personalizadas
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(101, activation='softmax')(x)  # 101 clases en Food101

model = Model(inputs=base_model.input, outputs=predictions)

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [ ]:
# Preprocesamiento específico para InceptionV3
datagen_train = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.inception_v3.preprocess_input,
    rescale=1./255,  # Normaliza las imágenes
    rotation_range=40,  # Aumenta la variabilidad de la orientación de las imágenes
    width_shift_range=0.2,  # Permite mover las imágenes horizontalmente
    height_shift_range=0.2,  # Permite mover las imágenes verticalmente
    shear_range=0.2,  # Rotación arbitraria
    zoom_range=0.2,  # Zoom aleatorio
    horizontal_flip=True,  # Permite reflejar horizontalmente las imágenes
    fill_mode='nearest'  # Cómo rellenar los píxeles vacíos al realizar transformaciones
    )

datagen_val = ImageDataGenerator(preprocessing_function=tf.keras.applications.inception_v3.preprocess_input)

train_generator = datagen_train.flow_from_directory(
    '/content/food101_split/train',
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical'
)

val_generator = datagen_val.flow_from_directory(
    '/content/food101_split/test',
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical'
)


Found 75750 images belonging to 101 classes.
Found 25250 images belonging to 101 classes.


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint_path = '/content/drive/MyDrive/model_checkpoints/inceptionv3_food101.h5'

checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

callbacks = [checkpoint, early_stop]

In [ ]:
history = model.fit(
    train_generator,
    epochs=30,  # Se detendrá antes si no mejora por 5 épocas
    validation_data=val_generator,
    callbacks=callbacks
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 0s 764ms/step - accuracy: 0.2458 - loss: 3.3360
Epoch 1: val_accuracy improved from -inf to 0.54764, saving model to /content/drive/MyDrive/model_checkpoints/inceptionv3_food101.h5


2368/2368 ━━━━━━━━━━━━━━━━━━━━ 1987s 830ms/step - accuracy: 0.2458 - loss: 3.3357 - val_accuracy: 0.5476 - val_loss: 1.7400
Epoch 2/30
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 0s 786ms/step - accuracy: 0.4658 - loss: 2.1258
Epoch 2: val_accuracy improved from 0.54764 to 0.57881, saving model to /content/drive/MyDrive/model_checkpoints/inceptionv3_food101.h5


2368/2368 ━━━━━━━━━━━━━━━━━━━━ 2004s 846ms/step - accuracy: 0.4658 - loss: 2.1258 - val_accuracy: 0.5788 - val_loss: 1.5895
Epoch 3/30
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 0s 773ms/step - accuracy: 0.5000 - loss: 1.9703
Epoch 3: val_accuracy improved from 0.57881 to 0.60103, saving model to /content/drive/MyDrive/model_checkpoints/inceptionv3_food101.h5


2368/2368 ━━━━━━━━━━━━━━━━━━━━ 1977s 835ms/step - accuracy: 0.5000 - loss: 1.9703 - val_accuracy: 0.6010 - val_loss: 1.5025
Epoch 4/30
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 0s 780ms/step - accuracy: 0.5096 - loss: 1.9162
Epoch 4: val_accuracy improved from 0.60103 to 0.61125, saving model to /content/drive/MyDrive/model_checkpoints/inceptionv3_food101.h5


2368/2368 ━━━━━━━━━━━━━━━━━━━━ 2050s 866ms/step - accuracy: 0.5096 - loss: 1.9162 - val_accuracy: 0.6112 - val_loss: 1.4513
Epoch 5/30
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 0s 767ms/step - accuracy: 0.5219 - loss: 1.8611
Epoch 5: val_accuracy improved from 0.61125 to 0.61937, saving model to /content/drive/MyDrive/model_checkpoints/inceptionv3_food101.h5


2368/2368 ━━━━━━━━━━━━━━━━━━━━ 1973s 828ms/step - accuracy: 0.5219 - loss: 1.8611 - val_accuracy: 0.6194 - val_loss: 1.4213
Epoch 6/30
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 0s 755ms/step - accuracy: 0.5284 - loss: 1.8304
Epoch 6: val_accuracy improved from 0.61937 to 0.62297, saving model to /content/drive/MyDrive/model_checkpoints/inceptionv3_food101.h5


2368/2368 ━━━━━━━━━━━━━━━━━━━━ 1926s 814ms/step - accuracy: 0.5284 - loss: 1.8304 - val_accuracy: 0.6230 - val_loss: 1.4015
Epoch 7/30
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 0s 751ms/step - accuracy: 0.5347 - loss: 1.8055
Epoch 7: val_accuracy improved from 0.62297 to 0.62689, saving model to /content/drive/MyDrive/model_checkpoints/inceptionv3_food101.h5


2368/2368 ━━━━━━━━━━━━━━━━━━━━ 1936s 811ms/step - accuracy: 0.5347 - loss: 1.8055 - val_accuracy: 0.6269 - val_loss: 1.3747
Epoch 8/30
 656/2368 ━━━━━━━━━━━━━━━━━━━━ 21:21 749ms/step - accuracy: 0.5453 - loss: 1.7440

In [ ]:
model.save('/content/drive/MyDrive/model_final_inceptionv3_food101.h5')

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('/content/drive/MyDrive/model_final_inceptionv3_food101.h5')